In [2]:
import pandas as pd
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df0 = pd.read_csv('/content/reviews_0-250.csv')
df1 = pd.read_csv('/content/reviews_1250-end.csv')
df2 = pd.read_csv('/content/reviews_250-500.csv')
df3 = pd.read_csv('/content/reviews_500-750.csv')
df4 = pd.read_csv('/content/reviews_750-1250.csv')

<ipython-input-3-e7c14bb6833d>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv('/content/reviews_0-250.csv')
<ipython-input-3-e7c14bb6833d>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/content/reviews_1250-end.csv')
<ipython-input-3-e7c14bb6833d>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('/content/reviews_750-1250.csv')


In [4]:
df_combined = pd.concat([df0, df1, df2, df3, df4], ignore_index=True)

In [5]:
df_combined['review_text'] = df_combined['review_text'].astype(str)

In [6]:
sampled_df = df_combined.sample(frac=0.1, random_state=1)

In [10]:
sampled_df = df_combined.sample(n=1000, random_state=1)

In [11]:
texts = [text.split() for text in sampled_df['review_text']]

In [16]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS

In [17]:
stop_words = set(stopwords.words('english')).union(STOPWORDS)
lemmatizer = WordNetLemmatizer()

In [18]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

In [19]:
sampled_df['cleaned_text'] = sampled_df['review_text'].apply(preprocess_text)
texts = [text.split() for text in sampled_df['cleaned_text']]

In [28]:
from gensim import corpora

dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

In [23]:
from gensim.models.ldamodel import LdaModel

In [24]:
num_topics = 10
passes = 20

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.388778693211581


In [14]:
for i, topic in ldamodel.show_topics(formatted=True, num_topics=num_topics, num_words=4):
    print(f"Topic #{i+1}: {topic}\n")

Topic #1: 0.002*"point" + 0.002*"great" + 0.001*"the" + 0.001*"non"

Topic #2: 0.006*"and" + 0.006*"this" + 0.004*"the" + 0.004*"I"

Topic #3: 0.042*"I" + 0.032*"and" + 0.029*"the" + 0.021*"to"

Topic #4: 0.052*"I" + 0.036*"and" + 0.026*"the" + 0.025*"my"

Topic #5: 0.032*"a" + 0.029*"and" + 0.028*"I" + 0.026*"it"



In [26]:
from gensim.models import TfidfModel

# 创建TF-IDF模型
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 使用TF-IDF重训练LDA模型
ldamodel_tfidf = LdaModel(corpus=corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=passes)

# 评估TF-IDF版本的模型
coherence_model_lda_tfidf = CoherenceModel(model=ldamodel_tfidf, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_tfidf = coherence_model_lda_tfidf.get_coherence()
print('\nCoherence Score with TF-IDF: ', coherence_lda_tfidf)


Coherence Score with TF-IDF:  0.380445940016631


In [27]:
from gensim.models.coherencemodel import CoherenceModel

# 计算一致性分数
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.40740606506851185
